In [1]:
from skimage import transform, filters, exposure
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.cross_validation import train_test_split
from keras.layers.convolutional import Conv2D

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import os
import glob
import pandas as pd
import math
from PIL import Image
import numpy as np
from scipy.misc import imread, imsave, imresize
from natsort import natsorted
from scipy.misc import imread
import random
random.seed( 42 ) 

nb_classes=62
# Path of data files
path = "/home/ivan/python/julia/data"
 
# Input image dimensions
img_rows, img_cols = 20, 20
 
 

### Images preprocessing ###

for setType in ["test","train"]:
    # We have to make sure files are sorted according to labels, even if they don't have trailing zeros
    files = natsorted(glob.glob(path + "/"+setType+"/*"))
    """
    if setType == 'test':
        X_test = np.array([np.array(Image.open(fname)) for fname in files])
    else:
        X_train = np.array([np.array(Image.open(fname)) for fname in files])
    """
    if setType == 'test':
        X_test = np.array([np.array(imread(fname, flatten=True)) for fname in files])
    else:
        X_train = np.array([np.array(imread(fname, flatten=True)) for fname in files])
X_test=X_test[:,:,:,np.newaxis]
X_train=X_train[:,:,:,np.newaxis]
X_train /= 255
X_test /= 255
X_test.shape
    

(6220, 20, 20, 1)

In [3]:
def label2int(ch):
    asciiVal = ord(ch)
    if(asciiVal<=57): #0-9
        asciiVal-=48
    elif(asciiVal<=90): #A-Z
        asciiVal-=55
    else: #a-z
        asciiVal-=61
    return asciiVal
    
def int2label(i):
    if(i<=9): #0-9
        i+=48
    elif(i<=35): #A-Z
        i+=55
    else: #a-z
        i+=61
    return chr(i)
# Load labels
y_train = pd.read_csv("/home/ivan/python/julia/trainLabels.csv").values[:,1] #Keep only label
 
# Convert labels to one-hot vectors
Y_train = np.zeros((y_train.shape[0], len(np.unique(y_train))))
 
for i in range(y_train.shape[0]):
    Y_train[i][label2int(y_train[i])] = 1 # One-hot
Y_train_all = Y_train.copy()
X_train_all = X_train.copy()

In [4]:
X_train, X_val, Y_train, Y_val = \
        train_test_split(X_train_all, Y_train_all, test_size=0.2, stratify=np.argmax(Y_train_all, axis=1))

In [7]:
from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout, concatenate, Conv2DTranspose
inputs = Input((img_rows, img_cols, 1))
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv3), conv2], axis=3)
conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv1], axis=3)
conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)

"""
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
"""

flat = Flatten()(conv7)
dense1 = Dense(2048, activation="relu")(flat)
drp1 = Dropout(0.5)(dense1)
dense2 = Dense(2048, activation="relu")(drp1)
drp2 = Dropout(0.5)(dense2)
dense3 = Dense(nb_classes, activation="softmax")(drp1)

model = Model(input=inputs, output=dense3)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [8]:
### LEARNING ###
from keras.optimizers import Adam, Adadelta
# First, use AdaDelta for some epochs because AdaMax gets stuck
model.compile(loss='categorical_crossentropy', 
              optimizer=Adadelta(),  
              metrics=["accuracy"])

In [ ]:
# 20 epochs is sufficient
model.fit(X_train, Y_train, batch_size=128,
                    #nb_epoch=20, 
                    epochs=20,
                    validation_data=(X_val, Y_val),
                    verbose=1)

Train on 5026 samples, validate on 1257 samples
Epoch 1/20
5026/5026 [==============================] - 284s - loss: 3.8626 - acc: 0.0609 - val_loss: 3.7924 - val_acc: 0.0573
Epoch 2/20
1664/5026 [========>.....................] - ETA: 182s - loss: 3.7948 - acc: 0.0523

In [39]:
# Parametrize the image augmentation class
datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    shear_range = 0.4,
    zoom_range = 0.3,                    
    channel_shift_range = 0.1)

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adamax',  
              metrics=["accuracy"])

# We want to keep the best model. This callback will store 
# in a file the weights of the model with the highest validation accuracy  
saveBestModel = ModelCheckpoint("best.kerasModelWeights", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True)

# Make the model learn using the image generator
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=128),
                samples_per_epoch=len(X_train),
                epochs=500, 
                validation_data=(X_val, Y_val),
                callbacks=[saveBestModel],
                verbose=1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=500, steps_per_epoch=39, callbacks=[<keras.ca..., validation_data=(array([[[..., verbose=1)`
  from ipykernel import kernelapp as app


Epoch 1/500
39/39 [==============================] - 6s - loss: 3.7842 - acc: 0.0725 - val_loss: 3.7662 - val_acc: 0.0732
Epoch 2/500
39/39 [==============================] - 5s - loss: 3.7721 - acc: 0.0711 - val_loss: 3.7663 - val_acc: 0.0732
Epoch 3/500
39/39 [==============================] - 5s - loss: 3.7788 - acc: 0.0713 - val_loss: 3.7652 - val_acc: 0.0732
Epoch 4/500
39/39 [==============================] - 5s - loss: 3.7738 - acc: 0.0693 - val_loss: 3.7651 - val_acc: 0.0732
Epoch 5/500
39/39 [==============================] - 5s - loss: 3.7672 - acc: 0.0726 - val_loss: 3.7649 - val_acc: 0.0732
Epoch 6/500
39/39 [==============================] - 5s - loss: 3.7788 - acc: 0.0726 - val_loss: 3.7646 - val_acc: 0.0732
Epoch 7/500
39/39 [==============================] - 5s - loss: 3.7689 - acc: 0.0731 - val_loss: 3.7645 - val_acc: 0.0732
Epoch 8/500
39/39 [==============================] - 5s - loss: 3.7780 - acc: 0.0711 - val_loss: 3.7644 - val_acc: 0.0732
Epoch 9/500
39/39 [=====

39/39 [==============================] - 5s - loss: 3.6078 - acc: 0.1122 - val_loss: 3.4187 - val_acc: 0.1527
Epoch 65/500
39/39 [==============================] - 5s - loss: 3.6345 - acc: 0.1185 - val_loss: 3.3999 - val_acc: 0.1512
Epoch 66/500
39/39 [==============================] - 5s - loss: 3.6117 - acc: 0.1086 - val_loss: 3.3971 - val_acc: 0.1480
Epoch 67/500
39/39 [==============================] - 5s - loss: 3.6035 - acc: 0.1133 - val_loss: 3.3981 - val_acc: 0.1472
Epoch 68/500
39/39 [==============================] - 5s - loss: 3.6062 - acc: 0.1164 - val_loss: 3.3820 - val_acc: 0.1432
Epoch 69/500
39/39 [==============================] - 5s - loss: 3.6022 - acc: 0.1146 - val_loss: 3.3812 - val_acc: 0.1575
Epoch 70/500
39/39 [==============================] - 5s - loss: 3.5980 - acc: 0.1100 - val_loss: 3.3727 - val_acc: 0.1424
Epoch 71/500
39/39 [==============================] - 5s - loss: 3.5984 - acc: 0.1137 - val_loss: 3.3683 - val_acc: 0.1480
Epoch 72/500
39/39 [=========

39/39 [==============================] - 5s - loss: 3.5505 - acc: 0.1231 - val_loss: 3.2989 - val_acc: 0.1893
Epoch 97/500
39/39 [==============================] - 5s - loss: 3.5188 - acc: 0.1216 - val_loss: 3.2723 - val_acc: 0.1862
Epoch 98/500
19/39 [=============>................] - ETA: 2s - loss: 3.5525 - acc: 0.1348

In [12]:
# Predict the class (give the index in the one-hot vector of the most probable class)
Y_test_pred = model.predict_classes(X_test)

# Translate integers to character labels
vInt2label = np.vectorize(int2label)
Y_test_pred = vInt2label(Y_test_pred)

# Save the predicitions in Kaggle format
np.savetxt(path+"/CNN_pred.csv", np.c_[range(6284,len(Y_test_pred)+6284),Y_test_pred], delimiter=',', header = 'ID,Class', comments = '', fmt='%s')



6208/6220 [============================>.] - ETA: 0s

In [7]:
from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout, concatenate, Conv2DTranspose
inputs = Input((img_rows, img_cols, 1))
conv1 = Conv2D(32, (3, 3), activation="relu", padding="same")(inputs)
conv1 = Conv2D(32, (3, 3), activation="relu", padding="same")(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)


conv2 = Conv2D(64, (3, 3), activation="relu", padding="same")(pool1)
conv2 = Conv2D(64, (3, 3), activation="relu", padding="same")(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation="relu", padding="same")(pool2)
conv3 = Conv2D(128, (3, 3), activation="relu", padding="same")(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)


conv4 = Conv2D(256, (3, 3), activation="relu", padding="same")(pool3)
conv4 = Conv2D(256, (3, 3), activation="relu", padding="same")(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation="relu", padding="same")(pool4)
conv5 = Conv2D(512, (3, 3), activation="relu", padding="same")(conv5)

up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=3)
conv6 = Conv2D(256, (3, 3), activation="relu", padding="same")(up6)
conv6 = Conv2D(256, (3, 3), activation="relu", padding="same")(conv6)

up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=3)
conv7 = Conv2D(128, (3, 3), activation="relu", padding="same")(up7)
conv7 = Conv2D(128, (3, 3), activation="relu", padding="same")(conv7)

up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=3)
conv8 = Conv2D(64, (3, 3), activation="relu", padding="same")(up8)
conv8 = Conv2D(64, (3, 3), activation="relu", padding="same")(conv8)

up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=3)
conv9 = Conv2D(32, (3, 3), activation="relu", padding="same")(up9)
conv9 = Conv2D(32, (3, 3), activation="relu", padding="same")(conv9)




#conv10 = Convolution2D(nb_classes,1, activation='softmax')(conv5)
flat = Flatten()(conv9)
dense1 = Dense(2048, activation="relu")(flat)
drp1 = Dropout(0.5)(dense1)
dense2 = Dense(2048, activation="relu")(drp1)
drp2 = Dropout(0.5)(dense2)
dense3 = Dense(nb_classes, activation="softmax")(drp1)

model = Model(input=inputs, output=dense3)
#model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[ 'accuracy'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


ValueError: "concat" mode can only merge layers with matching output shapes except for the concat axis. Layer shapes: [(None, 4, 4, 256), (None, 5, 5, 128)]